In [13]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KDTree as skKNN
from cuml import KNN as cumlKNN
import cudf
import os

# Helper Functions

In [14]:
from timeit import default_timer

class Timer(object):
    def __init__(self):
        self._timer = default_timer
    
    def __enter__(self):
        self.start()
        return self

    def __exit__(self, *args):
        self.stop()

    def start(self):
        """Start the timer."""
        self.start = self._timer()

    def stop(self):
        """Stop the timer. Calculate the interval in seconds."""
        self.end = self._timer()
        self.interval = self.end - self.start

In [15]:
import gzip
def load_data(nrows, ncols, cached = 'data/mortgage.npy.gz',source='mortgage'):
    if os.path.exists(cached) and source=='mortgage':
        print('use mortgage data')
        with gzip.open(cached) as f:
            X = np.load(f)
        X = X[np.random.randint(0,X.shape[0]-1,nrows),:ncols]
    else:
        print('use random data')
        X = np.random.random((nrows,ncols)).astype('float32')
    df = pd.DataFrame({'fea%d'%i:X[:,i] for i in range(X.shape[1])}).fillna(0)
    return df

In [16]:
from sklearn.metrics import mean_squared_error
def array_equal(a,b,threshold=1e-2,with_sign=True,metric='mse'):
    a = to_nparray(a)
    b = to_nparray(b)
    if with_sign == False:
        a,b = np.abs(a),np.abs(b)
    if metric=='mse':
        error = mean_squared_error(a,b)
    else:
        error = np.sum(a!=b)/(a.shape[0]*a.shape[1])
    res = error<threshold
    return res

def to_nparray(x):
    if isinstance(x,np.ndarray) or isinstance(x,pd.DataFrame):
        return np.array(x)
    elif isinstance(x,np.float64):
        return np.array([x])
    elif isinstance(x,cudf.DataFrame) or isinstance(x,cudf.Series):
        return x.to_pandas().values
    return x    

# Run tests

In [17]:
%%time
nrows = 50
ncols = 40

X = load_data(nrows,ncols)
print('data',X.shape)

use mortgage data
data (50, 40)
CPU times: user 5.61 s, sys: 740 ms, total: 6.35 s
Wall time: 6.34 s


In [18]:
n_neighbors = 10

In [19]:
%%time
knn_sk = skKNN(X)
D_sk,I_sk = knn_sk.query(X,n_neighbors)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 959 µs


In [20]:
%%time
X = cudf.DataFrame.from_pandas(X)

CPU times: user 144 ms, sys: 4 ms, total: 148 ms
Wall time: 144 ms


In [21]:
%%time
knn_cuml = cumlKNN(n_gpus=1)
knn_cuml.fit(X)

CPU times: user 44 ms, sys: 144 ms, total: 188 ms
Wall time: 183 ms


In [22]:
D_cuml,I_cuml = knn_cuml.query(X,n_neighbors)

In [23]:
passed = array_equal(D_sk,D_cuml)
message = 'compare knn: cuml vs sklearn distances %s'%('equal'if passed else 'NOT equal')
print(message)
passed = array_equal(I_sk,I_cuml)
message = 'compare knn: cuml vs sklearn indexes %s'%('equal'if passed else 'NOT equal')
print(message)

compare knn: cuml vs sklearn distances NOT equal
compare knn: cuml vs sklearn indexes equal
